Dependencies

In [2]:
import yfinance as yf

import pandas as pd
import plotly.express as px
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


import warnings
warnings.filterwarnings('ignore')
pd.options.plotting.backend = "plotly"

In [3]:
nifty = yf.Ticker("^NSEI")
nifty = nifty.history(period="max")
nifty

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2007-09-17 00:00:00+05:30,4518.450195,4549.049805,4482.850098,4494.649902,0,0.0,0.0
2007-09-18 00:00:00+05:30,4494.100098,4551.799805,4481.549805,4546.200195,0,0.0,0.0
2007-09-19 00:00:00+05:30,4550.250000,4739.000000,4550.250000,4732.350098,0,0.0,0.0
2007-09-20 00:00:00+05:30,4734.850098,4760.850098,4721.149902,4747.549805,0,0.0,0.0
2007-09-21 00:00:00+05:30,4752.950195,4855.700195,4733.700195,4837.549805,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-08-09 00:00:00+05:30,24386.849609,24419.750000,24311.199219,24367.500000,215100,0.0,0.0
2024-08-12 00:00:00+05:30,24320.050781,24472.800781,24212.099609,24347.000000,279900,0.0,0.0
2024-08-13 00:00:00+05:30,24342.349609,24359.949219,24116.500000,24139.000000,239700,0.0,0.0


In [3]:
nifty.index = nifty.index.date
nifty.rename_axis('Date', inplace=True)
nifty

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2007-09-17,4518.450195,4549.049805,4482.850098,4494.649902,0,0.0,0.0
2007-09-18,4494.100098,4551.799805,4481.549805,4546.200195,0,0.0,0.0
2007-09-19,4550.250000,4739.000000,4550.250000,4732.350098,0,0.0,0.0
2007-09-20,4734.850098,4760.850098,4721.149902,4747.549805,0,0.0,0.0
2007-09-21,4752.950195,4855.700195,4733.700195,4837.549805,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-08-08,24248.550781,24340.500000,24079.699219,24117.000000,311900,0.0,0.0
2024-08-09,24386.849609,24419.750000,24311.199219,24367.500000,215100,0.0,0.0
2024-08-12,24320.050781,24472.800781,24212.099609,24347.000000,279900,0.0,0.0


## EDA & Cleaning the Data

In [4]:
nifty.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4144 entries, 2007-09-17 to 2024-08-14
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          4144 non-null   float64
 1   High          4144 non-null   float64
 2   Low           4144 non-null   float64
 3   Close         4144 non-null   float64
 4   Volume        4144 non-null   int64  
 5   Dividends     4144 non-null   float64
 6   Stock Splits  4144 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 259.0+ KB


In [5]:
nifty.describe()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
count,4144.000000,4144.000000,4144.000000,4144.000000,4.144000e+03,4144.0,4144.0
mean,9787.739973,9841.119813,9718.727635,9781.563323,2.033903e+05,0.0,0.0
std,5158.312577,5171.166230,5136.726496,5155.517221,2.099569e+05,0.0,0.0
min,2553.600098,2585.300049,2252.750000,2524.199951,0.000000e+00,0.0,0.0
25%,5561.687378,5598.775024,5513.350098,5554.000000,0.000000e+00,0.0,0.0
50%,8376.924805,8413.250000,8321.799805,8360.924805,1.749000e+05,0.0,0.0
75%,11865.812256,11919.524902,11792.787842,11844.962158,2.845250e+05,0.0,0.0
max,25030.949219,25078.300781,24956.400391,25010.900391,1.811000e+06,0.0,0.0


In [6]:
nifty.index = nifty.index.astype('datetime64[ns]')
nifty

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2007-09-17,4518.450195,4549.049805,4482.850098,4494.649902,0,0.0,0.0
2007-09-18,4494.100098,4551.799805,4481.549805,4546.200195,0,0.0,0.0
2007-09-19,4550.250000,4739.000000,4550.250000,4732.350098,0,0.0,0.0
2007-09-20,4734.850098,4760.850098,4721.149902,4747.549805,0,0.0,0.0
2007-09-21,4752.950195,4855.700195,4733.700195,4837.549805,0,0.0,0.0
...,...,...,...,...,...,...,...
2024-08-08,24248.550781,24340.500000,24079.699219,24117.000000,311900,0.0,0.0
2024-08-09,24386.849609,24419.750000,24311.199219,24367.500000,215100,0.0,0.0
2024-08-12,24320.050781,24472.800781,24212.099609,24347.000000,279900,0.0,0.0


In [7]:
nifty.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4144 entries, 2007-09-17 to 2024-08-14
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          4144 non-null   float64
 1   High          4144 non-null   float64
 2   Low           4144 non-null   float64
 3   Close         4144 non-null   float64
 4   Volume        4144 non-null   int64  
 5   Dividends     4144 non-null   float64
 6   Stock Splits  4144 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 259.0 KB


In [8]:
nifty = nifty.drop(columns=['Dividends','Stock Splits'])

In [9]:
nifty.plot.line(y=['Open','Close'], title='NIFTY through the years')

## Setting up the Target


In [10]:
nifty['Tommorrow'] = nifty['Close'].shift(-1)
nifty.dropna(inplace=True)
nifty['Target'] = (nifty['Tommorrow'] > nifty['Close']).astype(int)
nifty

,Open,High,Low,Close,Volume,Tommorrow,Target
Date,,,,,,,
2007-09-17,4518.450195,4549.049805,4482.850098,4494.649902,0,4546.200195,1
2007-09-18,4494.100098,4551.799805,4481.549805,4546.200195,0,4732.350098,1
2007-09-19,4550.250000,4739.000000,4550.250000,4732.350098,0,4747.549805,1
2007-09-20,4734.850098,4760.850098,4721.149902,4747.549805,0,4837.549805,1
2007-09-21,4752.950195,4855.700195,4733.700195,4837.549805,0,4932.200195,1
...,...,...,...,...,...,...,...
2024-08-07,24289.400391,24337.699219,24184.900391,24297.500000,317600,24117.000000,0
2024-08-08,24248.550781,24340.500000,24079.699219,24117.000000,311900,24367.500000,1
2024-08-09,24386.849609,24419.750000,24311.199219,24367.500000,215100,24347.000000,0


In [11]:
nifty = nifty[nifty['Volume']>0]

In [12]:
nifty

,Open,High,Low,Close,Volume,Tommorrow,Target
Date,,,,,,,
2013-01-21,6085.750000,6094.350098,6065.100098,6082.299805,130900,6048.500000,0
2013-01-22,6080.149902,6101.299805,6040.500000,6048.500000,129000,6054.299805,1
2013-01-23,6052.850098,6069.799805,6021.149902,6054.299805,137000,6019.350098,0
2013-01-24,6046.200195,6065.299805,6007.850098,6019.350098,185200,6074.649902,1
2013-01-25,6024.500000,6080.549805,6014.450195,6074.649902,147600,6074.799805,1
...,...,...,...,...,...,...,...
2024-08-07,24289.400391,24337.699219,24184.900391,24297.500000,317600,24117.000000,0
2024-08-08,24248.550781,24340.500000,24079.699219,24117.000000,311900,24367.500000,1
2024-08-09,24386.849609,24419.750000,24311.199219,24367.500000,215100,24347.000000,0


## Base Model

In [13]:
model = RandomForestClassifier(n_estimators=1000, 
                               min_samples_split=100, 
                               random_state=69)

In [14]:
# Split the data into train and test
trainDS = nifty.iloc[:-100]
testDS = nifty.iloc[-100:]

In [15]:
xi = trainDS.columns[:-2] # predictors

In [16]:
model.fit(trainDS[xi], trainDS['Target']) # train the model

RandomForestClassifier(min_samples_split=100, n_estimators=1000,
                       random_state=69)

## Evaluation

In [17]:
predictions =pd.Series(model.predict(testDS[xi]), 
                       index= testDS.index)

In [18]:
predictions

Date
2024-03-11    0
2024-03-12    1
2024-03-13    1
2024-03-14    1
2024-03-15    1
             ..
2024-08-07    0
2024-08-08    0
2024-08-09    0
2024-08-12    0
2024-08-13    0
Length: 100, dtype: int64

In [19]:
test = lambda func, preds=predictions, testDS=testDS: \
    round(float(func(testDS['Target'], preds)), 4) * 100

base_metrics = {
    'Accuracy': test(accuracy_score),
    'Precision': test(precision_score),
    'Recall': test(recall_score),
    'F1 Score': test(f1_score)}

In [20]:
fig = px.bar(x=base_metrics.keys(), y=base_metrics.values(), color=base_metrics.keys())
fig.update_layout(title='Model Metrics',
                  xaxis_title='Metric', 
                  yaxis_title='Value',
                  legend_title='Legend')
fig.show()

## Backtesting

In [21]:
# Bactesting the model and creating predictions

def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []
    all_tragets =[]
    seasons = range(start, data.shape[0], step)
    
    for i in seasons:
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        
        model.fit(train[predictors], train['Target'])
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        targets = test['Target']
        all_predictions.append(preds)
        all_tragets.append(targets)
        
    return pd.concat(all_predictions), pd.concat(all_tragets)

In [22]:
Bpreds, Btargets = backtest(nifty, model, xi)

In [23]:
def plot_metrics(predictions, testData):
    test = lambda func, preds=predictions, testData=testData: \
        round(func(testData, preds), 4)

    metrics = {
        'Accuracy': test(accuracy_score),
        'Precision': test(precision_score),
        'Recall': test(recall_score),
        'F1 Score': test(f1_score)}

    fig = px.bar(x=metrics.keys(), y=metrics.values(), color=metrics.keys())
    fig.update_layout(title='Model Metrics',
                      xaxis_title='Metric', 
                      yaxis_title='Value',
                      legend_title='Legend')

    # Add a threshold line at 0.8
    fig.add_hline(y=0.8, line_dash="dot", line_color="red")

    fig.show()

In [24]:
plot_metrics(Bpreds, Btargets)

## Improving the Model

In [25]:
horizons = [2,5,60,250,750]
newX = []

for horizon in horizons:
    rollingAVG = nifty.rolling(horizon).mean()
    
    colName = f'Rolling_Ratio_{horizon}'
    nifty[colName] = nifty['Close'] / rollingAVG['Close']
    
    col2Name = f"Trend_{horizon}"
    nifty[col2Name] = nifty.shift(1).rolling(horizon).sum()['Target']
    
    newX += [colName, col2Name]

nifty = nifty.dropna()    

In [26]:
nifty

,Open,High,Low,Close,Volume,Tommorrow,Target,Rolling_Ratio_2,Trend_2,Rolling_Ratio_5,Trend_5,Rolling_Ratio_60,Trend_60,Rolling_Ratio_250,Trend_250,Rolling_Ratio_750,Trend_750
Date,,,,,,,,,,,,,,,,,
2016-02-18,7177.399902,7215.100098,7127.850098,7191.750000,246700,7210.750000,1,1.005825,2.0,1.013141,4.0,0.947340,26.0,0.883852,116.0,0.996824,384.0
2016-02-19,7170.549805,7226.850098,7145.950195,7210.750000,192300,7234.549805,1,1.001319,2.0,1.009283,4.0,0.951176,27.0,0.886805,116.0,0.999243,385.0
2016-02-22,7208.850098,7252.399902,7200.700195,7234.549805,154400,7109.549805,0,1.001648,2.0,1.010588,4.0,0.955569,28.0,0.890379,116.0,1.002322,385.0
2016-02-23,7240.299805,7241.700195,7090.700195,7109.549805,194400,7018.700195,0,0.991286,1.0,0.991429,4.0,0.940662,27.0,0.875726,115.0,0.984806,385.0
2016-02-24,7075.000000,7090.799805,7009.750000,7018.700195,199700,6970.600098,0,0.993570,0.0,0.981216,3.0,0.930538,26.0,0.865299,114.0,0.972052,384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-07,24289.400391,24337.699219,24184.900391,24297.500000,317600,24117.000000,0,1.006315,1.0,0.995193,2.0,1.030548,36.0,1.131161,145.0,1.291976,413.0
2024-08-08,24248.550781,24340.500000,24079.699219,24117.000000,311900,24367.500000,1,0.996272,1.0,0.995087,1.0,1.021582,36.0,1.121844,145.0,1.281612,412.0
2024-08-09,24386.849609,24419.750000,24311.199219,24367.500000,215100,24347.000000,0,1.005167,1.0,1.008337,2.0,1.030440,37.0,1.132507,146.0,1.294140,412.0


### Updated  Model

In [27]:
model2 = RandomForestClassifier(n_estimators=1000,min_samples_split=100, random_state=69)

In [28]:
def backtest2(data, model, predictors, start=2500, step=250):
    all_predictions = []
    all_tragets =[]
    seasons = range(start, data.shape[0], step)
    
    for i in seasons:
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        
        model.fit(train[predictors], train['Target'])
        preds = model.predict_proba(test[predictors])[:,1]
        preds[preds>=0.6] = 1
        preds[preds<0.6] = 0
        
        preds = pd.Series(preds, index=test.index)
        
        
        targets = test['Target']
        
        all_predictions.append(preds)
        all_tragets.append(targets)
        
    return pd.concat(all_predictions), pd.concat(all_tragets)

In [29]:
preds2, targets2 = backtest2(nifty, model2, newX, start=250)

In [30]:
plot_metrics(preds2, targets2)

## Conclusion

Backtested Base model works better than others and can be used for 'long' position trades
- As it has almost 80% precision - which means its able to predict Positive movement in nifty about 80% of the time 
- And has a Type 1 Error (False Positives) about than 20%